In [21]:
trials_filename = '../corpora/test_utt_list_2/data/vctk_dev_trials_f/trials'
scores_filename = '../exp/result/results-2024-01-02-15-43-10/ASV-vctk_dev_enrolls_anon-vctk_dev_trials_f_anon/scores'
scores_file = open(scores_filename, 'r').readlines()
trials_file = open(trials_filename, 'r').readlines()

c_miss=1
c_fa=1
p_target=0.01


scores = [] # 分数
labels = [] # 由0和1组成

trials = {}
for line in trials_file:
    utt1, utt2, target = line.rstrip().split()
    trial = utt1 + " " + utt2
    trials[trial] = target
    
for line in scores_file:
    utt1, utt2, score = line.rstrip().split()
    trial = utt1 + " " + utt2
    if trial in trials:
        scores.append(float(score))
        if trials[trial] == "target":
            labels.append(1)
        else:
            labels.append(0)
    else:
        raise Exception("Missing entry for " + utt1 + " and " + utt2
            + " " + scores_filename)
        
print(len(scores))
print(len(labels))

101
101


In [22]:
# compute error rates
# input: scores, labels
# output: fnrs, fprs, thresholds
from operator import itemgetter


# 创建假阴性率列表、假阳性率列表和给出这些误差率的判定阈值列表
def ComputeErrorRates(scores, labels):
    # sort the scores from smallest to largest, and also get the corresponding indexes of the sorted scores
    # treat the sorted scores as the thresholds at which the error-rates are evaluated
    # scores升序排序，将排序后的scores作为评估误差率的临界值
    sorted_indexes, thresholds = zip(*sorted(
        [(index, threshold) for index, threshold in enumerate(scores)],
        key=itemgetter(1)
    ))
    
    labels = [labels[i] for i in sorted_indexes]
    fnrs = []
    fprs = []

    # 循环结束时，fnrs[i]是由于错误地拒绝了小于thresholds[i]的score而造成的错误数量
    # fprs[i]是正确接受score大于thresholds[i]的总次数
    for i in range(0, len(labels)):
        if i == 0:
            fnrs.append(labels[i])
            fprs.append(1 - labels[i])
        else:
            fnrs.append(fnrs[i-1] + labels[i])
            fprs.append(fprs[i-1] + 1 - labels[i])
    fnrs_norm = sum(labels)
    # print(f"sum labels = {fnrs[len(labels) - 1]}, fnrs_norm={fnrs_norm}")
    # fnrs_norm = fnrs[len(lables) - 1]
    # fnrs_norm统计的是labels中1的个数
    # fprs_norm统计的是labels中0的个数
    # fnrs[i]是指labels[0]~labels[1]中1的个数
    fprs_norm = len(labels) - fnrs_norm
    # false negative rates
    fnrs = [x / float(fnrs_norm) for x in fnrs]

    # Divide by the total number of corret positives to get the
    # true positive rate.  Subtract these quantities from 1 to
    # get the false positive rates.
    # false positive rates
    fprs = [1 - x / float(fprs_norm) for x in fprs]

    print(f"fnrs, len: {len(fnrs)}, type: {type(fnrs)}")
    print(f"fprs, len: {len(fprs)}, type: {type(fprs)}")
    print(f"thresholds, len: {len(thresholds)}, type: {type(thresholds)}")
    return fnrs, fprs, thresholds
        

In [23]:
# ComputeMinDcf
# input: fnrs, fprs, thresholds, p_target, c_miss, c_fa
# output:min_dcf, min_c_det_threshold
# 计算检测成本函数的最小值
def ComputeMinDcf(fnrs, fprs, thresholds, p_target, c_miss, c_fa):
    min_c_det = float("inf")
    min_c_det_threshold = thresholds[0]
    for i in range(0, len(fnrs)):
        # See Equation (2).  it is a weighted sum of false negative
        # and false positive errors.
        c_det = c_miss * fnrs[i] * p_target + c_fa * fprs[i] * (1 - p_target)
        if c_det < min_c_det:
            min_c_det = c_det
            min_c_det_threshold = thresholds[i]
    # See Equations (3) and (4).  Now we normalize the cost.
    c_def = min(c_miss * p_target, c_fa * (1 - p_target))
    min_dcf = min_c_det / c_def

    print(f"min_dcf is {min_dcf}")
    print(f"min_c_det_threshold is {min_c_det_threshold}")
    return min_dcf, min_c_det_threshold

In [24]:
fnrs, fprs, thresholds = ComputeErrorRates(scores, labels)
mindcf, threshold = ComputeMinDcf(fnrs, fprs, thresholds, p_target,
         c_miss, c_fa)

print("{0:.4f}".format(mindcf))
print("minDCF is {0:.4f} at threshold {1:.4f} (p-target={2}, c-miss={3},"
    "c-fa={4})".format(mindcf, threshold, p_target,c_miss, c_fa))

sum labels = 34, fnrs_norm=34
fnrs, len: 101, type: <class 'list'>
fprs, len: 101, type: <class 'list'>
thresholds, len: 101, type: <class 'tuple'>
min_dcf is 0.11764705882352942
min_c_det_threshold is 47.62333
0.1176
minDCF is 0.1176 at threshold 47.6233 (p-target=0.01, c-miss=1,c-fa=1)


In [3]:
import numpy
# 无穷，浮点型，无确切值
print(numpy.inf)
type(numpy.inf)
t = numpy.inf
t

inf


inf